In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
sst_col_list = [c for c in add_train.columns if 'sst' in c]
sst_col_list

['sst-2010-1',
 'sst-2010-2',
 'sst-2010-3',
 'sst-2010-4',
 'sst-2010-5',
 'sst-2010-6',
 'sst-2010-7',
 'sst-2010-8',
 'sst-2010-9',
 'sst-2010-10']

In [53]:
ice_col_list = [c for c in add_train.columns if 'ice' in c]
ice_col_list

['icec-2010-1',
 'icec-2010-2',
 'icec-2010-3',
 'icec-2010-4',
 'icec-2010-5',
 'icec-2010-6',
 'icec-2010-7',
 'icec-2010-8',
 'icec-2010-9',
 'icec-2010-10']

In [74]:
add_test.groupby(['loc_group'])['climateregions__climateregion'].unique()

loc_group
0      [0]
1      [4]
2      [4]
3      [4]
4      [4]
      ... 
509    [9]
510    [9]
511    [9]
512    [9]
513    [9]
Name: climateregions__climateregion, Length: 514, dtype: object

In [72]:
add_test.groupby(['loc_group'])['climateregions__climateregion']

loc_group
0      61
1      61
2      61
3      61
4      61
       ..
509    61
510    61
511    61
512    61
513    61
Name: climateregions__climateregion, Length: 514, dtype: int64

In [77]:
print(len(add_train['lat'].unique()), len(add_train['lon'].unique()))

23 31


In [78]:
add_train['lat'].unique()

array([0.        , 0.04545455, 0.09090909, 0.13636364, 0.18181818,
       0.22727273, 0.27272727, 0.31818182, 0.36363636, 0.40909091,
       0.45454545, 0.5       , 0.54545455, 0.59090909, 0.63636364,
       0.68181818, 0.72727273, 0.77272727, 0.81818182, 0.86363636,
       0.90909091, 0.95454545, 1.        ])

In [81]:
add_train.groupby(['lat', 'lon'])[target[0]].mean().reset_index().sort_values(by=['lat', 'lon'])

,lat,lon,contest-tmp2m-14d__tmp2m
0,0.000000,0.833333,23.474873
1,0.000000,0.866667,23.427205
2,0.045455,0.833333,23.170019
3,0.045455,0.866667,22.525948
4,0.045455,0.900000,22.475270
...,...,...,...
509,1.000000,0.733333,5.182245
510,1.000000,0.766667,5.431043
511,1.000000,0.800000,4.736491
512,1.000000,0.833333,4.268409


In [79]:
add_train['lon'].unique()

array([0.83333333, 0.86666667, 0.9       , 0.8       , 0.93333333,
       0.66666667, 0.7       , 0.73333333, 0.76666667, 0.96666667,
       1.        , 0.63333333, 0.36666667, 0.4       , 0.43333333,
       0.46666667, 0.5       , 0.53333333, 0.56666667, 0.6       ,
       0.23333333, 0.26666667, 0.3       , 0.33333333, 0.2       ,
       0.13333333, 0.16666667, 0.1       , 0.06666667, 0.03333333,
       0.        ])

In [4]:
train_df = pd.read_csv('./fe_v2_train_076.csv')
test_df = pd.read_csv('./fe_v2_test_076.csv')

In [5]:
exclude_cols = ['index', 'startdate']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 129


In [7]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [8]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5']

In [9]:
print("Len:", len(best_cols))

Len: 116


In [10]:
def assign_train_test_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df

def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

def create_feature(df, alias_name, col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", col_name, f"first_{alias_name}_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", col_name, f"min_{alias_name}_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", col_name, f"max_{alias_name}_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_first'] = df[col_name] - df[f"first_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_min'] = df[col_name] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_max'] = df[col_name] - df[f"max_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_max')
    
    df[f'range_{alias_name}_{suffix_num}'] = df[f"max_{alias_name}_{suffix_num}"] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'range_{alias_name}_{suffix_num}')
    
    df[f'scale_{alias_name}_{suffix_num}'] = (df[col_name] - df[f"min_{alias_name}_{suffix_num}"]) / df[f'range_{alias_name}_{suffix_num}']
    df = assign_train_test_df(df, f'scale_{alias_name}_{suffix_num}')
    
    return [df, f'diff_{alias_name}_{suffix_num}_first',
   f'diff_{alias_name}_{suffix_num}_min',
    f'diff_{alias_name}_{suffix_num}_max',
    f'range_{alias_name}_{suffix_num}', f'scale_{alias_name}_{suffix_num}']

In [51]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_10 = 'contest-wind-h10-14d__wind-hgt-10'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'
prate_col = 'nmme-prate-34w__nmmemean'
wind_u_15_col = 'wind-uwnd-925-2010-15'
wind_u_18_col = 'wind-uwnd-925-2010-18'
wind_v_250_3_col = 'wind-vwnd-250-2010-3'
wind_important = 'contest-wind-h500-14d__wind-hgt-500'

In [12]:
train_df[sst_col_list] = add_train[sst_col_list]
test_df[sst_col_list] = add_test[sst_col_list]

In [13]:
df = pd.concat([train_df[main_attrs], test_df[main_attrs]], axis=0)
df.shape

(407088, 129)

In [14]:
pres_res = create_feature(df, 'pres', pres_col, 1, season_group_cols)
df = pres_res[0]
pres_feature = pres_res[1:]

Nan
Nan
Nan


In [15]:
slp_res = create_feature(df, 'slp', slp_col, 1, season_group_cols)
df = slp_res[0]
slp_feature = slp_res[1:]

Nan
Nan
Nan


In [16]:
precip_res = create_feature(df, 'precip', precip_col, 1, season_group_cols)
df = precip_res[0]
precip_feature = precip_res[1:]

Nan
Nan
Nan


In [17]:
sst_1_month_res = create_feature(df, 'sst_1_month', sst_1_col, 1, group_cols)
df = sst_1_month_res[0]
sst_1_month_feature = sst_1_month_res[1:]

Nan
Nan
Nan


In [18]:
sst_5_month_res = create_feature(df, 'sst_5_month', sst_5_col, 1, group_cols)
df = sst_5_month_res[0]
sst_5_month_feature = sst_5_month_res[1:]

Nan
Nan
Nan


In [19]:
sst_3_month_res = create_feature(df, 'sst_3_month', sst_3_col, 1, group_cols)
df = sst_3_month_res[0]
sst_3_month_feature = sst_3_month_res[1:]

Nan
Nan
Nan


In [20]:
sst_4_month_res = create_feature(df, 'sst_4_month', sst_4_col, 1, group_cols)
df = sst_4_month_res[0]
sst_4_month_feature = sst_4_month_res[1:]

Nan
Nan
Nan


In [21]:
sst_6_month_res = create_feature(df, 'sst_6_month', sst_6_col, 1, group_cols)
df = sst_6_month_res[0]
sst_6_month_feature = sst_6_month_res[1:]

Nan
Nan
Nan


In [22]:
wind_10_res = create_feature(df, 'wind_10', wind_col_10, 1, season_group_cols)
df = wind_10_res[0]
wind_10_feature = wind_10_res[1:]

Nan
Nan
Nan


In [23]:
wind_10_res_month = create_feature(df, 'wind_10_month', wind_col_10, 1, group_cols)
df = wind_10_res_month[0]
wind_10_month_feature = wind_10_res_month[1:]

Nan
Nan
Nan


In [24]:
ccsm3_res_month_res = create_feature(df, 'ccsm3_month', ccsm3_col, 1, group_cols)
df = ccsm3_res_month_res[0]
ccsm3_month_feature = ccsm3_res_month_res[1:]

Nan
Nan
Nan


In [40]:
ccsm3_res = create_feature(df, 'ccsm3', ccsm3_col, 1, season_group_cols)
df = ccsm3_res[0]
ccsm3_feature = ccsm3_res[1:]

Nan
Nan
Nan


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [35]:
prate_res_month_res = create_feature(df, 'prate_month', prate_col, 1, group_cols)
df = prate_res_month_res[0]
prate_month_feature = prate_res_month_res[1:]

Nan
Nan
Nan


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [47]:
wind_u_15_res_month_res = create_feature(df, 'wind_u_15_month', wind_u_15_col, 1, group_cols)
df = wind_u_15_res_month_res[0]
wind_u_15_month_feature = wind_u_15_res_month_res[1:]

Nan
Nan
Nan


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [25]:
modified_best_cols = [c for c in best_cols]

In [26]:
df['sst_1_lag_1'] = df.groupby(group_cols)[sst_1_col].shift(1)
df = assign_train_test_df(df, 'sst_1_lag_1')

df['sst_1_lag_2'] = df.groupby(group_cols)[sst_1_col].shift(2)
df = assign_train_test_df(df, 'sst_1_lag_2')

df['sst_1_lag_3'] = df.groupby(group_cols)[sst_1_col].shift(3)
df = assign_train_test_df(df, 'sst_1_lag_3')

df['sst_5_lag_1'] = df.groupby(group_cols)[sst_5_col].shift(1)
df = assign_train_test_df(df, 'sst_5_lag_1')

df['sst_3_lag_1'] = df.groupby(group_cols)[sst_3_col].shift(1)
df = assign_train_test_df(df, 'sst_3_lag_1')

df['sst_4_lag_1'] = df.groupby(group_cols)[sst_4_col].shift(1)
df = assign_train_test_df(df, 'sst_4_lag_1')

df['sst_6_lag_1'] = df.groupby(group_cols)[sst_6_col].shift(1)
df = assign_train_test_df(df, 'sst_6_lag_1')


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [27]:
df['sst_1_lag_1_season'] = df.groupby(season_group_cols)[sst_1_col].shift(1)
df = assign_train_test_df(df, 'sst_1_lag_1_season')
df['sst_5_lag_1_season'] = df.groupby(season_group_cols)[sst_5_col].shift(1)
df = assign_train_test_df(df, 'sst_5_lag_1_season')

df['sst_4_lag_1_season'] = df.groupby(season_group_cols)[sst_4_col].shift(1)
df = assign_train_test_df(df, 'sst_4_lag_1_season')

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fr

In [28]:
# shift(lag_days).rolling(lag_days).mean()

In [29]:
# df['sst_1_rolling_1'] = df.groupby(group_cols)[sst_1_col].shift(1).rolling(1).mean()
# df = assign_train_test_df(df, 'sst_1_rolling_1')

df['sst_5_rolling_1'] = df.groupby(group_cols)[sst_5_col].shift(1).rolling(1).mean()
df = assign_train_test_df(df, 'sst_5_rolling_1')

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [48]:
features = modified_best_cols + pres_feature + wind_u_15_month_feature + ccsm3_month_feature + sst_1_month_feature + ['sst_1_lag_1', 'sst_5_lag_1', 'sst_4_lag_1', 'sst_1_lag_1_season', 'sst_5_lag_1_season', 
                                                                     'sst_4_lag_1_season']

In [42]:
test_df['index'] = add_test['index']

In [43]:
# train_df[features + target].to_csv('best_train.csv', index=False)
# test_df[features + ['index']].to_csv('best_test.csv', index=False)

In [50]:
features

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [49]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
    
   
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        
        clf = CatBoostRegressor(verbose=200, task_type="GPU",
                           devices='0:1', 
                
                                        iterations=5000,
                                        use_best_model=True,
                                       
                                        od_type='iter',
                                        od_wait=50,
                                      )
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v10.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:01,  3.13it/s]


(375734, 142) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 3423.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 57.6ms	remaining: 4m 48s
200:	learn: 1.1760468	test: 1.1818377	best: 1.1818377 (200)	total: 11.8s	remaining: 4m 42s
400:	learn: 0.9085981	test: 0.9181534	best: 0.9181534 (400)	total: 22.7s	remaining: 4m 20s
600:	learn: 0.7847603	test: 0.7959277	best: 0.7959277 (600)	total: 34s	remaining: 4m 8s
800:	learn: 0.7018742	test: 0.7149773	best: 0.7149773 (800)	total: 47.3s	remaining: 4m 7s
1000:	learn: 0.6425029	test: 0.6568397	best: 0.6568397 (1000)	total: 58.7s	remaining: 3m 54s
1200:	learn: 0.5981353	test: 0.6134525	best: 0.6134525 (1200)	total: 1m 10s	remaining: 3m 41s
1400:	learn: 0.5614097	test: 0.5779157	best: 0.5779157 (1400)	total: 1m 21s	remaining: 3m 29s
1600:	learn: 0.5327531	test: 0.5502827	best: 0.5502827 (1600)	total: 1m 33s	remaining: 3m 18s
1800:	learn: 0.5072888	test: 0.5256050	best: 0.5256050 (1800)	total: 1m 36s	remaining: 2m 50s
2000:	learn: 0.4859266	test: 0.5051571	best: 0.505157

 20%|█████████████████▍                                                                     | 1/5 [04:15<17:00, 255.21s/it]Warning: less than 75% gpu memory available for training. Free: 3423.5625 Total: 24268.3125


...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 24.8ms	remaining: 2m 4s
200:	learn: 1.1687191	test: 1.1837023	best: 1.1837023 (200)	total: 12.2s	remaining: 4m 50s
400:	learn: 0.9182600	test: 0.9333887	best: 0.9333887 (400)	total: 23.5s	remaining: 4m 29s
600:	learn: 0.7871354	test: 0.8017218	best: 0.8017218 (600)	total: 35.4s	remaining: 4m 19s
800:	learn: 0.7033155	test: 0.7184028	best: 0.7184028 (800)	total: 46.9s	remaining: 4m 6s
1000:	learn: 0.6426033	test: 0.6587082	best: 0.6587082 (1000)	total: 59.4s	remaining: 3m 57s
1200:	learn: 0.5975320	test: 0.6142107	best: 0.6142107 (1200)	total: 1m 11s	remaining: 3m 47s
1400:	learn: 0.5618398	test: 0.5793606	best: 0.5793606 (1400)	total: 1m 23s	remaining: 3m 35s
1600:	learn: 0.5322214	test: 0.5506682	best: 0.5506682 (1600)	total: 1m 35s	remaining: 3m 22s
1800:	learn: 0.5081946	test: 0.5274060	best: 0.5274060 (1800)	total: 1m 46s	remaining: 3m 9s
2000:	learn: 0.4855691	test: 0.5054256	best: 0.50542

 80%|█████████████████████████████████████████████████████████████████████▌                 | 4/5 [18:37<04:45, 285.42s/it]Warning: less than 75% gpu memory available for training. Free: 3423.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 38.7ms	remaining: 3m 13s
200:	learn: 1.1664991	test: 1.1713875	best: 1.1713875 (200)	total: 12.2s	remaining: 4m 52s
400:	learn: 0.9061701	test: 0.9135559	best: 0.9135559 (400)	total: 24.6s	remaining: 4m 42s
600:	learn: 0.7821713	test: 0.7912711	best: 0.7912711 (600)	total: 36.6s	remaining: 4m 27s
800:	learn: 0.6948045	test: 0.7059088	best: 0.7059088 (800)	total: 48.6s	remaining: 4m 14s
1000:	learn: 0.6346690	test: 0.6472426	best: 0.6472426 (1000)	total: 1m	remaining: 4m 2s
1200:	learn: 0.5909687	test: 0.6048464	best: 0.6048464 (1200)	total: 1m 12s	remaining: 3m 48s
1400:	learn: 0.5570974	test: 0.5720013	best: 0.5720013 (1400)	total: 1m 24s	remaining: 3m 35s
1600:	learn: 0.5291576	test: 0.5451149	best: 0.5451149 (1600)	total: 1m 34s	remaining: 3m 21s
1800:	learn: 0.5044536	test: 0.5216983	best: 0.5216983 (1800)	total: 1m 46s	remaining: 3m 9s
2000:	learn: 0.4827365	test: 0.5010491	best: 0.5010491

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 13.40it/s]


cos_sim with best submission: 0.9999045967745356
